### gmsh - Flow over polygon (cylinder) script

In [48]:
import gmsh
import os
import subprocess
import numpy as np
import sys
import meshio

def flow_over_cyli():

    gmsh.initialize()
    gmsh.model.add("gmsh_flow_over_cyli")

    inner_rad = 1; outer_rad = 2; angle = 45; angle_rad = np.radians(angle); origin = 0;
    lc = 5; lc_boundary = 5e-2;
    a = 30; b = 30;

    gmsh.model.geo.addPoint(a, b, origin, lc, 1)
    gmsh.model.geo.addPoint(-a, b, origin, lc, 2)
    gmsh.model.geo.addPoint(-a, -b, origin, lc, 3)
    gmsh.model.geo.addPoint(a, -b, origin, lc, 4)

    gmsh.model.geo.addPoint(a, -b//10, origin, lc, 5)
    gmsh.model.geo.addPoint(-a//5, -b//10, origin, lc, 6)
    gmsh.model.geo.addPoint(-a//5, b//10, origin, lc, 7)
    gmsh.model.geo.addPoint(a, b//10, origin, lc, 8)

    gmsh.model.geo.addLine(1, 2, 9)
    gmsh.model.geo.addLine(2, 3, 10)
    gmsh.model.geo.addLine(3, 4, 11)
    gmsh.model.geo.addLine(4, 5, 12)
    gmsh.model.geo.addLine(5, 6, 13)
    gmsh.model.geo.addLine(6, 7, 14)
    gmsh.model.geo.addLine(7, 8, 15)
    gmsh.model.geo.addLine(8, 1, 16)
    gmsh.model.geo.addLine(5, 8, 17)  

    gmsh.model.geo.addPoint(np.cos(angle_rad)*inner_rad, np.sin(angle_rad)*inner_rad, 0, lc, 18)
    gmsh.model.geo.addPoint(-np.cos(angle_rad)*inner_rad, np.sin(angle_rad)*inner_rad, 0, lc, 19)
    gmsh.model.geo.addPoint(-np.cos(angle_rad)*inner_rad, -np.sin(angle_rad)*inner_rad, 0, lc, 20)
    gmsh.model.geo.addPoint(np.cos(angle_rad)*inner_rad, -np.sin(angle_rad)*inner_rad, 0, lc, 21)
    gmsh.model.geo.addPoint(origin, origin, origin, lc, 22)

    gmsh.model.geo.addCircleArc(18, 22, 19, 23)
    gmsh.model.geo.addCircleArc(19, 22, 20, 24)
    gmsh.model.geo.addCircleArc(20, 22, 21, 25)
    gmsh.model.geo.addCircleArc(21, 22, 18, 26)

    gmsh.model.geo.addCurveLoop([23, 24, 25, 26], 27)
    gmsh.model.geo.addCurveLoop([9, 10, 11, 12, 13, 14, 15, 16], 28)
    gmsh.model.geo.addCurveLoop([-15, -14, -13, 17], 29)
    
    gmsh.model.geo.addPlaneSurface([28], 30)
    gmsh.model.geo.addPlaneSurface([29, 27], 31)

    gmsh.model.geo.synchronize()
    gmsh.model.geo.mesh.setTransfiniteCurve(12, 25, "Progression", coef=1.05)  # Curve 12 with geometric progression
    gmsh.model.geo.mesh.setTransfiniteCurve(16, 25, "Progression", coef=1.05) # Curve 16 with geometric progression


    #
    # SizeMax -                     /------------------
    #                              /
    #                             /
    #                            /
    # SizeMin -o----------------/
    #          |                |    |
    #        Point         DistMin  DistMax

    gmsh.model.mesh.field.add("Distance", 1)
    # gmsh.model.mesh.field.setNumbers(1, "PointsList", [7, 6])
    gmsh.model.mesh.field.setNumbers(1, "CurvesList", [23, 24, 25, 26])
    gmsh.model.mesh.field.setNumber(1, "Sampling", 100)

    gmsh.model.mesh.field.add("Threshold", 2)
    gmsh.model.mesh.field.setNumber(2, "InField", 1)
    gmsh.model.mesh.field.setNumber(2, "SizeMin", lc / 150)
    gmsh.model.mesh.field.setNumber(2, "SizeMax", lc / 5)
    gmsh.model.mesh.field.setNumber(2, "DistMin", 0)
    gmsh.model.mesh.field.setNumber(2, "DistMax", 5)

    gmsh.model.mesh.field.add("Distance", 3)
    gmsh.model.mesh.field.setNumbers(3, "CurvesList", [15, 14, 13])
    gmsh.model.mesh.field.setNumber(3, "Sampling", 100)

    gmsh.model.mesh.field.add("Threshold", 4)
    gmsh.model.mesh.field.setNumber(4, "InField", 3)
    gmsh.model.mesh.field.setNumber(4, "SizeMin", lc / 40)
    gmsh.model.mesh.field.setNumber(4, "SizeMax", lc / 2)
    gmsh.model.mesh.field.setNumber(4, "DistMin", 0)
    gmsh.model.mesh.field.setNumber(4, "DistMax", 5)

    gmsh.model.mesh.field.add("Box", 5)
    gmsh.model.mesh.field.setNumber(5, "VIn", lc / 40)
    gmsh.model.mesh.field.setNumber(5, "VOut", lc )
    gmsh.model.mesh.field.setNumber(5, "XMin", inner_rad) 
    gmsh.model.mesh.field.setNumber(5, "XMax", a/2)
    gmsh.model.mesh.field.setNumber(5, "YMin", -b/10)
    gmsh.model.mesh.field.setNumber(5, "YMax", b/10)

    gmsh.model.mesh.field.add("Box", 6)
    gmsh.model.mesh.field.setNumber(6, "VIn", lc / 75)
    gmsh.model.mesh.field.setNumber(6, "VOut", lc )
    gmsh.model.mesh.field.setNumber(6, "XMin", -a/5) 
    gmsh.model.mesh.field.setNumber(6, "XMax", -inner_rad+0.2)
    gmsh.model.mesh.field.setNumber(6, "YMin", -(inner_rad+0.2))
    gmsh.model.mesh.field.setNumber(6, "YMax", inner_rad+0.2)


    gmsh.model.mesh.field.add("Min", 7)
    gmsh.model.mesh.field.setNumbers(7, "FieldsList", [2, 4, 5, 6])
    gmsh.model.mesh.field.setAsBackgroundMesh(7)

    gmsh.option.setNumber("Mesh.MeshSizeExtendFromBoundary", 0)
    gmsh.option.setNumber("Mesh.MeshSizeFromPoints", 0)
    gmsh.option.setNumber("Mesh.MeshSizeFromCurvature", 0)



    gmsh.option.setNumber("Mesh.Algorithm", 5)
    # gmsh.model.mesh.generate(2)

    # Extrusion
    layers = 1; extrude_height = 1; extrude_vector = [0, 0, extrude_height];
    ex = gmsh.model.geo.extrude([(2, 30), (2, 31)], extrude_vector[0], extrude_vector[1], extrude_vector[2], numElements = [layers], recombine = True)

    gmsh.model.geo.synchronize()

    # Physical Surfaces
    surface_ids = sorted([e[1] for e in ex if e[0] == 2])
    volume_ids = [e[1] for e in ex if e[0] == 3]

    # Print surface and volume IDs for verification
    print("Extruded surface IDs: ", surface_ids)
    print("Volume IDs: ", volume_ids)

    gmsh.model.addPhysicalGroup(3, volume_ids, 1)
    gmsh.model.setPhysicalName(3, 1, "Fluid")

    gmsh.model.addPhysicalGroup(2, surface_ids[0:3], 2)
    gmsh.model.setPhysicalName(2, 2, "Inlet")
    gmsh.model.addPhysicalGroup(2, [56, 72, 98], 3)
    gmsh.model.setPhysicalName(2, 3, "Outlet")
    gmsh.model.addPhysicalGroup(2, [30, 31, 73, 115], 4)
    gmsh.model.setPhysicalName(2, 4, "Sides")
    gmsh.model.addPhysicalGroup(2, [102, 106, 110, 114], 5)
    gmsh.model.setPhysicalName(2, 5, "Cylinder")


    # Generating mesh
    print(ex)
    gmsh.model.mesh.generate(3)

    mesh_file = os.path.abspath("gmsh_flow_over_cyli.msh2")
    gmsh.write(mesh_file)

    # Launch the GUI to see the results:
    if '-nopopup' not in sys.argv:
        gmsh.fltk.run()

    gmsh.finalize()

    # Saving the mesh file, and opening it
    mesh = meshio.read(mesh_file)

    # Extract only the necessary parts of the mesh to avoid issues with cell sets
    cells = [(cell_block.type, cell_block.data) for cell_block in mesh.cells]

    # Create a new mesh object without cell sets
    mesh_clean = meshio.Mesh(points=mesh.points, cells=cells)

    meshio.write("gmsh_flow_over_cyli.vtk", mesh_clean)


if __name__ == "__main__":
    flow_over_cyli()



Extruded surface IDs:  [44, 48, 52, 56, 60, 60, 64, 64, 68, 68, 72, 73, 98, 102, 106, 110, 114, 115]
Volume IDs:  [1, 2]
[(2, 73), (3, 1), (2, 44), (2, 48), (2, 52), (2, 56), (2, 60), (2, 64), (2, 68), (2, 72), (2, 115), (3, 2), (2, 68), (2, 64), (2, 60), (2, 98), (2, 102), (2, 106), (2, 110), (2, 114)]
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 9 (Line)
Info    : [ 10%] Meshing curve 10 (Line)
Info    : [ 10%] Meshing curve 11 (Line)
Info    : [ 10%] Meshing curve 12 (Line)
Info    : [ 20%] Meshing curve 13 (Line)
Info    : [ 20%] Meshing curve 14 (Line)
Info    : [ 20%] Meshing curve 15 (Line)
Info    : [ 20%] Meshing curve 16 (Line)
Info    : [ 30%] Meshing curve 17 (Line)
Info    : [ 30%] Meshing curve 23 (Circle)
Info    : [ 30%] Meshing curve 24 (Circle)
Info    : [ 30%] Meshing curve 25 (Circle)
Info    : [ 40%] Meshing curve 26 (Circle)
Info    : [ 40%] Meshing curve 33 (Extruded)
Info    : [ 40%] Meshing curve 34 (Extruded)
Info    : [ 40%] Meshing curve 35 (Extrud

ReadError: Could not deduce file format from path '/home/affu5154/OpenFOAM/affu5154-9/run/pygmsh/Mesh_gmsh/gmsh_flow_over_cyli.msh2'.